<a href="https://colab.research.google.com/github/datvodinh10/project-DD/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
!pip install timm

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip /content/gdrive/MyDrive/Handwritten_OCR/training_data.zip

In [ ]:
import torch

In [ ]:
!git clone https://github.com/datvodinh10/project-DD.git
%cd project-DD

In [ ]:
from src.model.trainer import Trainer

In [ ]:
%cd ..

## Train

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
config = {
    'transformer':{
        'embed_size': 384,
        'num_heads':6,
        'num_layers':2,
        'max_len': 10_000,
        'dropout':0.2,
        'bias':False,
        'block_size':256,

    },
    'encoder':{
        'type': 'swin_transformer' # {'resnet18','resnet50,'vgg','swin_transformer'}
    },
    'device':device,
    'lr':1e-4,
    'batch_size':32,
    'num_epochs':5
}

In [ ]:
trainer = Trainer(config      = config,
                  IMAGE_PATH  = "/content/new_train",
                  TARGET_PATH = "/content/gdrive/MyDrive/Handwritten_OCR/train_gt.txt")

In [ ]:
trainer.train()

In [ ]:
save_dict = {
    'state_dict':trainer.model.state_dict(),
    'config':config,
    'vocab_size':trainer.vocabulary.vocab_size
}
file_path = '/content/model.pth'
torch.save(save_dict, file_path)

## Predict

In [ ]:
from src.utils.inference import Inference
from src.utils.transform import Transform
from PIL import Image
import os
import torch

In [ ]:
infer = Inference(DATA_PATH   = file_path,
                  TARGET_PATH = "/content/gdrive/MyDrive/Handwritten_OCR/train_gt.txt")

In [ ]:
idx = torch.randint(1000,())
img = Image.open(os.path.join("/content/new_train",os.listdir("/content/new_train")[idx]))
trainer.vocabulary.decode(infer.predict(Transform.train_transform(img).unsqueeze(0).to(device))[0])

In [ ]:
from torchvision import models
models.resnet18